In [1]:
%load_ext autoreload
%autoreload 2

from lastfm import *
from prep_data import *
from text_classes import *
from tf_graph import *

# Collect wikipedia summary of artists/bands

In [2]:
artist = 'The Doors'

# If generate mongodb collection
#linked_artists = get_top_n_linked_artists(artist, n_min=150, match_min=0.1, n_per_artist=15)
#collection = dump_wikipedia_summary_to_mongodb(artist, linked_artists)

# If use existing mongodb collection
collection = get_collection(artist)

# Prep training text

In [3]:
batch_size = 150
n_char_per_memory = 50
n_char_random_offset_max = 10

text = get_concat_summary(collection)
train = Text(text)
train.prep_for_training(batch_size, n_char_per_memory, n_char_random_offset_max)

print('length of each part: {}'.format(train.len_part_list))

length of each part: [1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1592]


# Build LSTM graph

In [4]:
cnfg = Config()
cnfg.chars = train.chars
cnfg.char2id = train.char2id
cnfg.id2char = train.id2char
cnfg.n_char_per_memory = train.n_char_per_memory
cnfg.n_char = len(train.chars)

cnfg.lstm_state_sizes = [100, 200]
cnfg.lr = 0.1

lstm_text_generator = LSTMTextGenerator(cnfg)

# Train!

In [5]:
cnfg.max_step = 1000000
cnfg.dropout_keep_prob = 0.75
cnfg.generate_every = 10000
cnfg.n_generate = 200
cnfg.log_save_as = 'logs/test_lstm.log'
cnfg.ckp_dir = 'ckp/test_lstm'

test_text = 'Hello Kitty is '
test = Text(test_text)

lstm_text_generator.train_model(cnfg, train, test)

Training starts @ 12/28/2017 03:46:05
Step 1 ends @ 12/28/2017 03:46:10 [Logloss] 4.542 [Accuracy] 0.0%
----------------------------------------------------------------------------------------------------
Hello Kitty is '0]–i.smàCIfäf'FÇa3 ’XH2ni2P—”[.!(UU!IERwnSzo

Bh$P3mU8QWp,(m"N:WDLxPv4fsM’-G8(à;]AB?)'sF712əs7vGq2:k9uh"”E"E”ÇZFD2O7G'b&nraYy"??bN4v–YÉTsLw;t(!Btn5vtC–nRTJJ.U7sSÉK],S1L$kTv”fR2àj.6əé-é””:ékmhiVW#bé)2C
Step 10,000 ends @ 12/28/2017 04:08:45 [Logloss] 1.962 [Accuracy] 0.4%
----------------------------------------------------------------------------------------------------
Hello Kitty is Andazd's Blanaén Amertsr, Alachif shngly his the Do. Toe he mous iccal-ul fluees Cues fom he d CoEpst Harn Capes ame the Happs fores colirocal.

Pigh Bover hecre group whhe Joonartund, hes-stedixksived
Step 20,000 ends @ 12/28/2017 04:45:27 [Logloss] 1.883 [Accuracy] 0.5%
----------------------------------------------------------------------------------------------------
Hello Kitty is We

KeyboardInterrupt: 